In [2]:
import glob
import tqdm
import re
import random

In [8]:
def clean_corpus(input_path,output_path):
    fin = open(input_path, "rt",encoding = 'utf-8')
    fout = open(output_path, "wt",encoding = 'utf-8')
    for line in fin:
        
        
        line = line.replace("of ", "")
        line = line.replace("body ", "")
        line = line.replace("img ", "")
        line = line.replace("% max-width ", "")
        line = line.replace("% div center page-break-after always", "")
        line = line.replace(" } ", "")
        fout.write(line)
        #break
        #fout.write(re.sub('\s+',' ',line))
    fin.close()
    fout.close()

In [9]:
clean_corpus("russian_nofraglit_corpus.txt","Russian_corpus.txt")

In [3]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
from IPython.display import clear_output
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle
import json
import pandas as pd 
from gensim.models import Word2Vec
from graph_utils import *

In [13]:
class Vectorizer():
    def __init__(self, corp_path):
        self.corp_path = corp_path
    
    def upload_corp(self):
        with open(self.corp_path, 'r',encoding = 'utf-8') as f:
            self.corp = list(set(f.read().split('\n')[:-1]))
        #self.corp = [text.replace('$$$', ' ') for text in self.corp]
    
    
    def log(self, part):
        clear_output(wait=True)
        print(f'{part} is processing')
        
    def make_tf_idf_matrix(self, token_pattern=None, min_df = 1, max_df=None, use_idf = True):
        if token_pattern:
            self.tfidf = TfidfVectorizer(analyzer='word', min_df=min_df, token_pattern=token_pattern, use_idf=use_idf)
        else:
            self.tfidf = TfidfVectorizer(analyzer='word', min_df=min_df, use_idf=use_idf)
            
        self.W = self.tfidf.fit_transform(self.corp)
        self.words_list = self.tfidf.get_feature_names_out()
        
    def make_svd(self, output_folder, k=6 ):
#         self.u, self.sigma, self.vt = svds(self.W, k)
        
#         self.descending_order_of_inds = np.flip(np.argsort(self.sigma))
#         self.u = self.u[:,self.descending_order_of_inds]
#         self.vt = self.vt[self.descending_order_of_inds,:]
#         self.sigma = np.diag(self.sigma[self.descending_order_of_inds])
        self.u, self.sigma, self.vT = svds(self.W, k)
        self.descending_order_of_inds = np.argsort(-self.sigma)
        
        self.u = self.u[:, self.descending_order_of_inds]
        self.sigma = np.diag(self.sigma[self.descending_order_of_inds])
        self.vT = self.vT[self.descending_order_of_inds, :]

        #Checking that sizes are ok
        #assert self.sigma.shape == (k,)
        #assert swlf.vt.shape == (k, self.W.shape[1])
        #assert swlf.u.shape == (self.W.shape[0], k)
        print (self.W.shape)
        print (self.sigma.shape)
        print (self.vT.shape)
        print (self.u.shape)
        
        self.embedded_matrix = self.sigma@self.vT
        #self.embedded_matrix = np.dot(np.diag(self.sigma), self.vt).T
        self.words_embedding_dict = dict(zip(self.words_list, self.embedded_matrix.T))
        #self.words_embedding_dict = dict(zip(self.words_list, self.embedded_matrix))
        
#         with open(output_folder+'/' + str(k) + '_sigma_vt.npy', 'wb') as f:
#             #np.save(f, np.dot(np.diag(self.sigma), self.vt).T)
#             np.save(f, self.embedded_matrix.T)
#         with open(output_folder+'/' +  str(k) + '_sigma.npy', 'wb') as f:
#             np.save(f, self.sigma)
#         with open(output_folder+'/' +  str(k) + '_u.npy', 'wb') as f:
#             np.save(f, self.u)
#         with open(output_folder+'/' +  str(k) + '_vt.npy', 'wb') as f:
#             np.save(f, self.vT)
            
        self.save_word_embedding(k)
    
    
    def get_emb_dict(self):
        
        self.log('Upload')
        self.upload_corp()
        self.log('TfIdf')
        self.make_tf_idf_matrix()
        self.log('SVD')
        self.make_svd('./Matrixes')
        
        return self.words_embedding_dict
    
    def save_word_embedding(self, shape):
        
        dict_ = {}
        for key, value in self.words_embedding_dict.items():
            dict_[key] = value.tolist()
        save_name = "Russian_WordVectorDict_SVD" + str(shape) + ".json"
        with open(save_name, 'w') as f:
            json.dump(dict_, f)
            
    

In [14]:
vect = Vectorizer('./Russian_corpus.txt')
emb_dict = vect.get_emb_dict()

SVD is processing
(6423, 589723)
(6, 6)
(6, 589723)
(6423, 6)


In [15]:
def load_corpus(corp_path):
    with open(corp_path, 'r',encoding = 'utf-8') as f:
        corp = list(set(f.read().split('\n')[:-1]))
    #corp = [text.replace('$$$', ' ') for text in corp]
    corp = [text.split() for text in corp]
    return corp

In [16]:
documents = load_corpus('Russian_corpus.txt')

In [17]:
len(documents)

6423

In [18]:
dimension = 6
model = Word2Vec(sentences=documents, vector_size=dimension, min_count=1)
model.save("word2vec_Russian.model")

In [19]:
dictionary = {key : model.wv[key] for key in model.wv.key_to_index}

In [20]:
def save_dictionary(dictionary,shape):
    for key, value in dictionary.items():
        dictionary[key] = value.tolist()
    save_name = "Russian_WordVectorDict_CBOW" + str(shape) + ".json"
    with open(save_name, 'w') as f:
        json.dump(dictionary, f)

In [21]:
save_dictionary( dictionary, dimension)

In [22]:
corpus_path = './Russian_corpus.txt'

In [23]:
def convert_text_to_vector(text, dictionary, n, m):
    text = text.strip().split()
    text_vectors = []
    text_words = []

    for i in range(len(text) - n + 1):
        gram_vec = []
        words = []
        for word in text[i:i+n]:
            if word not in dictionary:
                gram_vec = []
                break
            vec_ = dictionary[word][:m]
            gram_vec.append(vec_)
            words.append(word)
        if len(gram_vec) != n or len(gram_vec[-1]) != m:
            continue
        text_vectors.append(np.array(gram_vec).flatten())
        text_words.append(' '.join(words))
    text_dict = dict(zip(text_words, text_vectors))
    return text_dict

In [28]:
def vectorize_corpus_with_ngrams(corpus_path: str,
                                   separator: str,
                                   word_vector_dict: dict, n : int , m : int  ) -> np.ndarray:
    colnames=[ 'Text'] 
    df_corpus = pd.read_csv(corpus_path,delimiter= "(", names =colnames, header=None)
    corpus = df_corpus['Text'].tolist()
    vectorized_corpus = list()
    for text_index, text in enumerate(tqdm(corpus)):
        vectorized_text = dict()
        if not text:
            continue
        vectorized_t = convert_text_to_vector(text, word_vector_dict,n,m)
        vectorized_text['document_index'] = text_index
        vectorized_text['vectorized_text'] = vectorized_t
        vectorized_corpus.append(vectorized_text)
    return np.array(vectorized_corpus, dtype=object)

In [29]:
#How long is the n-gram
n = 1
#How long is a vector for every word
m = 6


In [30]:
with open('Russian_WordVectorDict_CBOW6.json', encoding='utf-8') as f:
    emb_dic = json.load(f)
#print(CBOW_dic)

In [31]:
vectorized_corpus = vectorize_corpus_with_ngrams(corpus_path,
                                               ' ',
                                               emb_dic,n,int(m/n))

100%|██████████| 6429/6429 [02:47<00:00, 38.40it/s]


In [32]:
np.save("Russian_vectorized_corpus_CBOW_1gram_6", vectorized_corpus)

In [33]:
with open("Russian_WordVectorDict_SVD6.json", encoding='utf-8') as f:
    emb_dic = json.load(f)

In [34]:
vectorized_corpus = vectorize_corpus_with_ngrams(corpus_path,
                                               ' ',
                                               emb_dic,n,int(m/n))

100%|██████████| 6429/6429 [02:25<00:00, 44.13it/s] 


In [35]:
np.save("Russian_vectorized_corpus_SVD_1gram_6", vectorized_corpus)

In [4]:
At_VECTORIZED_CORPUS = "./Russian_vectorized_corpus_SVD_1gram_6.npy"
at_corpus = np.load(At_VECTORIZED_CORPUS, allow_pickle=True)
at_corpus = at_corpus.tolist()

In [5]:
len(at_corpus)

6429

In [6]:
n = int(len(at_corpus)/100)
# printing n elements from list
test = random.choices(at_corpus, k=n)
np.save("Russian_vectorized_corpus_sample1%_SVD_1gram_6", np.array(test))


In [7]:
At_VECTORIZED_CORPUS = "./Russian_vectorized_corpus_CBOW_1gram_6.npy"
at_corpus = np.load(At_VECTORIZED_CORPUS, allow_pickle=True)
at_corpus = at_corpus.tolist()

In [8]:
len(at_corpus)

6429

In [9]:
n = int(len(at_corpus)/100)
# printing n elements from list
test = random.choices(at_corpus, k=n)
np.save("Russian_vectorized_corpus_sample1%_CBOW_1gram_6", np.array(test))
